###**HW1.Clasification of ASL dataset**

### 1. Data preparation: data loader

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class ASL_Datasets(Dataset):
  def __init__(self, path, transform=None):
    super(ASL_Datasets, self).__init__()

    df = pd.read_csv(path)
    y = df['label']
    del df['label']
    x = df.values

    self.x_tensor = torch.tensor(x).float()/255.
    self.y_tensor = torch.tensor(y)

    self.transform = transform

  def __getitem__(self, index):
    return self.x_tensor[index], self.y_tensor[index]

  def __len__(self):
    return self.x_tensor.shape[0]

In [2]:
train_data = ASL_Datasets(path="sign_mnist_train.csv")
valid_data = ASL_Datasets(path="sign_mnist_valid.csv")

train_loader = DataLoader(dataset=train_data, batch_size=500, shuffle=True)

### 2. Constructing a neural network architecture

In [3]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 24)

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = self.leaky_relu(x)
        x = self.fc2(x)
        x = self.leaky_relu(x)
        x = self.fc3(x)
        x = self.leaky_relu(x)
        x = self.fc4(x)
        return x

### 3. Loss function and optimization method

In [4]:
net = Net()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

### 4. Training of the neural network

In [5]:
para = []
for epoch in range(50):
  loss_val = 0
  for itr, data in enumerate(train_loader):
    optimizer.zero_grad()
    inputs, labels = data

    pred = net(inputs)
    loss = loss_func(pred, labels)

    loss.backward()
    optimizer.step()

    loss_val += loss.item()

  if epoch%5 == 0:
    para.append(net.state_dict())

  print("Epoch: " + str(epoch+1) + ", Loss: " + str(loss_val))

Epoch: 1, Loss: 149.2593801021576
Epoch: 2, Loss: 101.73960340023041
Epoch: 3, Loss: 81.45613121986389
Epoch: 4, Loss: 64.70466166734695
Epoch: 5, Loss: 50.31499642133713
Epoch: 6, Loss: 38.899387538433075
Epoch: 7, Loss: 29.607853531837463
Epoch: 8, Loss: 23.217653393745422
Epoch: 9, Loss: 18.322135508060455
Epoch: 10, Loss: 14.283384442329407
Epoch: 11, Loss: 11.4513508528471
Epoch: 12, Loss: 8.570897556841373
Epoch: 13, Loss: 5.216282323002815
Epoch: 14, Loss: 4.3022436536848545
Epoch: 15, Loss: 3.833174489438534
Epoch: 16, Loss: 2.178116610273719
Epoch: 17, Loss: 1.56307384185493
Epoch: 18, Loss: 1.2737747319042683
Epoch: 19, Loss: 3.9300959203392267
Epoch: 20, Loss: 32.42537811398506
Epoch: 21, Loss: 3.716110359877348
Epoch: 22, Loss: 1.8332886788994074
Epoch: 23, Loss: 1.3367884121835232
Epoch: 24, Loss: 1.006831580772996
Epoch: 25, Loss: 0.7763749370351434
Epoch: 26, Loss: 0.6530352737754583
Epoch: 27, Loss: 0.5267303716391325
Epoch: 28, Loss: 0.4628718248568475
Epoch: 29, Loss:

### 5. Prediction and Evaluation for test set

In [6]:
print(len(para))

10


In [7]:
max_accu = 0  # 가장 높은 정확도를 저장할 변수
best_param = None  # 가장 정확도가 높은 모델의 파라미터를 저장할 변수

# 저장된 파라미터별로 성능을 확인하는 코드
for i, param in enumerate(para):
    # 저장된 파라미터 로드
    net.load_state_dict(param)

    # 테스트 데이터로 예측
    input_test = valid_data.x_tensor
    pred_test = net(input_test)
    pred_category = torch.argmax(pred_test, dim=1)

    label_test = valid_data.y_tensor
    accu = torch.mean((pred_category == label_test).float())

    # 각 파라미터(epoch)에 따른 성능 출력
    print(f"Test accuracy for saved model {i+1}: {accu.item()}")

    # 정확도가 더 높으면 저장
    if accu.item() > max_accu:
        max_accu = accu.item()
        best_param = param  # 가장 성능이 좋은 파라미터 저장

# 가장 좋은 파라미터를 net에 로드
if best_param is not None:
    net.load_state_dict(best_param)

Test accuracy for saved model 1: 0.7928053736686707
Test accuracy for saved model 2: 0.7928053736686707
Test accuracy for saved model 3: 0.7928053736686707
Test accuracy for saved model 4: 0.7928053736686707
Test accuracy for saved model 5: 0.7928053736686707
Test accuracy for saved model 6: 0.7928053736686707
Test accuracy for saved model 7: 0.7928053736686707
Test accuracy for saved model 8: 0.7928053736686707
Test accuracy for saved model 9: 0.7928053736686707
Test accuracy for saved model 10: 0.7928053736686707


In [8]:
input_test = valid_data.x_tensor
pred_test = net(input_test)
pred_category = torch.argmax(pred_test, dim=1)

label_test = valid_data.y_tensor
accu = torch.mean((pred_category == label_test).float())

print("Test accuracy:", accu)

Test accuracy: tensor(0.7928)


## 수정 내용

1. Layer 4개 784 -> 512 -> 256 -> 128 -> 24
2. Active function: Leak ReLU
3. 5 epoch 마다 파라미터 저장
   -> 최선의 파라미터 선택
